# Price data from db

In [1]:
import requests
import json
import numpy as np
import pymongo
import pandas as pd
import datetime
from bs4 import BeautifulSoup
from pprint import pprint
from dateutil.relativedelta import relativedelta
from pymongo import MongoClient
%matplotlib inline

### Link to Mongodb

In [2]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
price_db = client.bitcoinprice_db
priceCollection = price_db.CoindeskDailyCollection

In [3]:
# Set list to catch json file
price_array = []
for n in priceCollection.find():
    price_array.append(n)

### DBdata to Pandas df

In [4]:
# To pd.DataFrame
price_df =  pd.DataFrame(price_array)
price_df.head()

,Close,Date,High,Low,Open,_id
0,0.09,2010-07-18,0.09,0.06,0.05,59c0ff05a78b030ec4ea3376
1,0.08,2010-07-19,0.09,0.08,0.09,59c0ff05a78b030ec4ea3377
2,0.07,2010-07-20,0.08,0.07,0.08,59c0ff05a78b030ec4ea3378
3,0.08,2010-07-21,0.08,0.07,0.07,59c0ff05a78b030ec4ea3379
4,0.05,2010-07-22,0.08,0.05,0.08,59c0ff05a78b030ec4ea337a


In [5]:
# Del _id col
del price_df['_id']

In [6]:
# Re-order
price_df = price_df[['Date', 'Open', 'High', 'Low', 'Close']]

In [7]:
# Show
price_df.head()

,Date,Open,High,Low,Close
0,2010-07-18,0.05,0.09,0.06,0.09
1,2010-07-19,0.09,0.09,0.08,0.08
2,2010-07-20,0.08,0.08,0.07,0.07
3,2010-07-21,0.07,0.08,0.07,0.08
4,2010-07-22,0.08,0.08,0.05,0.05


# `#################################################################`

# 配對price和news

### 將Date 設為 index

In [8]:
#make a copy
price_df['Datee'] = price_df['Date']

In [9]:
# set index
price_df.set_index('Date',inplace=True)

### 日期標記方法

In [10]:
import talib

In [11]:
price_df['C_back1'] = price_df['Close'].shift(-1)
price_df['C_forward1'] = price_df['Close'].shift(1)

In [12]:
# 標記漲跌用
price_df['Tag'] = 0

### 進行標記

In [13]:
for index, row in price_df.iterrows():
    diff = row['C_forward1']/row['Close']
    price_df.set_value(index, 'Diff', diff)
    try:
        if diff >= 1.2:
            price_df.set_value(index, 'Tag', 1)
            #print('大漲', index, diff)
        elif diff <= 0.7:
            price_df.set_value(index, 'Tag', 2)
            #print('大跌', index,diff)
        else:
            price_df.set_value(index, 'Tag', 0)
    except:
        print('error', index)

### 輸出"有標記"的日期

In [14]:
df_tag = price_df[(price_df['Tag'] == 1) | (price_df['Tag'] == 2)]

In [15]:
df_tag.head()

,Open,High,Low,Close,Datee,C_back1,C_forward1,Tag,Diff
Date,,,,,,,,,
2010-07-22,0.08,0.08,0.05,0.05,2010-07-22,0.06,0.08,1,1.600000
2010-07-24,0.06,0.06,0.05,0.05,2010-07-24,0.05,0.06,1,1.200000
2010-11-06,0.26,0.50,0.24,0.39,2010-11-06,0.34,0.26,2,0.666667
2010-11-08,0.34,0.37,0.23,0.24,2010-11-08,0.21,0.34,1,1.416667
2010-11-16,0.27,0.27,0.22,0.22,2010-11-16,0.23,0.27,1,1.227273


### connect news_db

In [16]:
news_db = client.bitcoinnews_db
newsCollection = news_db.newsCollection

### 編號配對兩個df

In [17]:
news_collect = []
news_tag_collect = []
news_markdate_collect = []

### 取出時間對應的新聞

In [18]:
i = 1
for index, row in df_tag.iterrows():
    if row['Tag'] == 1:
        news_tag = str(i) + 'U'
    else:
        news_tag = str(i) + 'D'
    df_tag.set_value(index, 'Newstag', news_tag)
    start = row['Datee'] + datetime.timedelta(days=-5)
    end = row['Datee'] + datetime.timedelta(days=5)
    today = row['Datee']
    for item in newsCollection.find({'Date':{'$gte': start, '$lte': end}}):
        news_collect.append(item)
        news_tag_collect.append(news_tag)
        news_markdate_collect.append(today)
    i += 1

D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [19]:
news_df =  pd.DataFrame(news_collect)
news_df['Newstag'] = news_tag_collect
news_df['Markday'] = news_markdate_collect

In [20]:
del news_df['_id']

In [21]:
news_df['BorA'] = 0

In [22]:
for index, row in news_df.iterrows():
    if row['Date'] >= row['Markday']:
        news_df.set_value(index, ['BorA'], 'after')
    else:
        news_df.set_value(index, ['BorA'], 'before')

### check data

In [32]:
news_df.head()

,Content,Date,Link,Source,Title,Newstag,Markday,BorA
0,\n2008 年11 月 1 日，一位自稱中本聰 (Satoshi Nakamoto) 的...,2013-04-12 15:12:00,https://technews.tw/2013/04/12/review-of-bitcoin/,technews,回顧比特幣 (bitcoin) 風雲，這一切如何開始？,19U,2013-04-10,after
1,\n一枚比特幣在今年二月份的時候價值大約 25 美元，而今天它的價值是 185 美元。諾貝爾...,2013-04-10 14:09:00,https://finance.technews.tw/2013/04/10/krugman...,technews,諾貝爾經濟學家 Krugman 錯了嗎：比特幣熱愈演愈烈,19U,2013-04-10,after
2,\n2008 年11 月 1 日，一位自稱中本聰 (Satoshi Nakamoto) 的...,2013-04-12 15:12:00,https://technews.tw/2013/04/12/review-of-bitcoin/,technews,回顧比特幣 (bitcoin) 風雲，這一切如何開始？,20U,2013-04-11,after
3,\n一枚比特幣在今年二月份的時候價值大約 25 美元，而今天它的價值是 185 美元。諾貝爾...,2013-04-10 14:09:00,https://finance.technews.tw/2013/04/10/krugman...,technews,諾貝爾經濟學家 Krugman 錯了嗎：比特幣熱愈演愈烈,20U,2013-04-11,before
4,\n2008 年11 月 1 日，一位自稱中本聰 (Satoshi Nakamoto) 的...,2013-04-12 15:12:00,https://technews.tw/2013/04/12/review-of-bitcoin/,technews,回顧比特幣 (bitcoin) 風雲，這一切如何開始？,21U,2013-04-13,before


In [30]:
# df_tag.head()

# 文本分析

In [31]:
#encoding=utf-8
import jieba

In [33]:
words = jieba.cut(news_df['Title'], cut_all=False)

In [35]:
for word in words:
    print(word)

AttributeError: 'Series' object has no attribute 'decode'

# `#################################################################`

### 畫圖、指標等

In [25]:
# price_df['Close: 30 Day Mean'] = price_df['Close'].rolling(window=20).mean()
# price_df['Upper'] = price_price_df['Close: 30 Day Mean'] + 2*price_df['Close'].rolling(window=20).std()
# price_df['Lower'] = price_df['Close: 30 Day Mean'] - 2*price_df['Close'].rolling(window=20).std()
# price_df[['Close','Close: 30 Day Mean','Upper','Lower']].plot(figsize=(16,6))

### Mongodb Query 筆記

#### df_tag.loc['2010-07-24', 'Before_news']

http://blog.csdn.net/buleriver/article/details/8912407

<pre>
大於：$gt
大於等於：$gte
小於：$lt
小於等於：$lte
不等於: $ne
IN：$in
NOT IN: $nin
取餘：$mod
非：$NOT
或：$OR
</pre>
<pre>
例子1：在XXX中查詢年齡>20且<=30的人：
[javascript] view plain copy 
db.XXX.find({ "age" :{ "$gt" :20,  "lte" :30}});  

例子2：在XXX中查詢年齡不是20的人：
[javascript] view plain copy 
db.XXX.find({ "age" :{ "$ne" :20}});  

例子3：名稱為BuleRiver或者BuleRiver2的人，不要使用OR
[javascript] view plain copy 
db.XXX.find({ "name" :{ "$in" :[ "BuleRiver" ,  "BuleRiver2" ]}});  

例子4：年齡MOD5等於3和不等於3的所有人：
[javascript] view plain copy 
db.XXX.find({ "age" :{ "$mod" :[5,3]}});  
db.XXX.find({ "age" :{ "$not" :{ "$mod" :[5, 3]}}});  

例子5：使用OR實現例子3
[javascript] view plain copy 
db.XXX.find({ "$OR" :[{ "name" : "BuleRiver" }, { "name" : "BuleRiver2" }]});  
</pre>

### 連結DB/with user, password的func筆記

In [26]:
# def _connect_mongo(host, port, username, password, db):
#     if username and password:
#         mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
#         conn = MongoClient(mongo_uri)
#     else:
#         conn = MongoClient(host, port)
#     return conn[db]

# def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
#     # Connect to MongoDB
#     db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)
#     # Make a query to the specific DB and Collection
#     cursor = db[collection].find(query)
#     # Expand the cursor and construct the DataFrame
#     df =  pd.DataFrame(list(cursor))
#     # Delete the _id
#     if no_id:
#         del df['_id']
#     return df